# Clean Datasets and create a master copy

We have 3 datasets of varying quality and completeness. We will clean them and create a master copy for further analysis.

We can use Pandas to read the datasets into dataframes.

Read all the datasets into dataframes, clean them and create a master copy.

In [9]:
import pandas as pd
from pathlib import Path

def clean_datasets(raw_players_extended_file, raw_players_file, raw_squads_file):
    """
    Reads and cleans the raw datasets.

    Parameters:
    raw_players_extended_file (str): Filepath of the raw players extended dataset.
    raw_players_file (str): Filepath of the raw players dataset.
    raw_squads_file (str): Filepath of the raw squads dataset.

    Returns:
    pandas.DataFrame: Cleaned players extended dataset.
    pandas.DataFrame: Cleaned players dataset.
    pandas.DataFrame: Cleaned squads dataset.
    """

    raw_players_extended = pd.read_csv(raw_players_extended_file, encoding="utf-8", low_memory=False)
    raw_players = pd.read_excel(raw_players_file)
    raw_squads = pd.read_excel(raw_squads_file)

    return raw_players_extended, raw_players, raw_squads

raw_players_file = Path("../raw_datasets/FIFA PLAYERS.xlsx")
raw_players_extended_file = Path("../raw_datasets/FIFA PLAYERS.csv")
raw_squads_file = Path("../raw_datasets/FIFA SQUADS.xlsx")

cleaned_players_extended, cleaned_players, cleaned_squads = clean_datasets(raw_players_extended_file, raw_players_file, raw_squads_file)

One of the datasets has been decoded using utf-8 encoding. This has resulted in the columns under "Values" and "Wages" as currency symbols. We will clean this dataset. Ideally we want them to be integers so that it is easier to perform calculations and analysis.

In [10]:
def convert_currency(value):
    """
    Convert a currency value to an integer.

    Parameters:
    value (str): The currency value to be converted.

    Returns:
    int: The converted currency value as an integer.
    """
    if value == '€0':
        return 0
    elif value[-1] == 'M':
        return int(float(value[1:-1]) * 1000000)
    elif value[-1] == 'K':
        return int(float(value[1:-1]) * 1000)
    else:
        return int(value)

cleaned_players_extended['Value'] = cleaned_players_extended['Value'].apply(convert_currency)
cleaned_players_extended['Wage'] = cleaned_players_extended['Wage'].apply(convert_currency)


Merge cleaned datasets to create a master dataset.

In [11]:
master = pd.merge(cleaned_players_extended, cleaned_players, left_on=["Name","Age", "Club", "Nationality", "Overall", "Potential", "Value", "Wage"], right_on=["Name","Age", "Club", "Nationality", "Overall", "Potential", "Value", "Wage"], how="left")
master = pd.merge(master, cleaned_squads, left_on=["Name","Age", "Club"], right_on=["Player", "Age", "Club"], how="left")

Clean the datasets by dropping columns with all NaN values and the "Unnamed: 0" column.

In [12]:
master.dropna(axis=1, how='all', inplace=True)
master.drop("Unnamed: 0", axis=1, inplace=True)


Saves the 'master' dataframe to an Excel file named 'master.xlsx'.

In [13]:
master.to_excel(Path("master.xlsx").resolve(), index=False)

In [14]:
import plotly.graph_objects as go

# Create a scatter plot
fig = go.Figure(data=go.Scatter(x=master['Name'], y=master['Age'], mode='markers'))

# Set the title and axis labels
fig.update_layout(title='Age Distribution', xaxis_title='Name', yaxis_title='Age')

# Show the plot
fig.show()


In [15]:
import plotly.graph_objects as go

# Create a scatter plot
fig = go.Figure(data=go.Scatter(x=master['Age'], y=master['Nationality'], mode='markers', text=master['Flag'], marker=dict(size=10)))

# Set the title and axis labels
fig.update_layout(title='Age vs Nationality', xaxis_title='Age', yaxis_title='Nationality')

# Show the plot
fig.show()


In [16]:
# import matplotlib.pyplot as plt

# # Group the data by Club and calculate the mean Overall value
# club_overall = master.groupby('Club')['Overall'].mean()

# # Create a bar plot
# plt.figure(figsize=(12, 6))
# club_overall.plot(kind='bar')
# plt.title('Overall Value by Club')
# plt.xlabel('Club')
# plt.ylabel('Overall Value')
# plt.xticks(rotation=90)
# plt.show()
